In [ ]:
import numpy as np 
import pandas as pd 
import os
import sys 
import scipy 
from scipy import stats
from astropy.io import fits #allows python to interpurt fits data files 
from glob import glob # Unix style pathname pattern expansion
from matplotlib import pyplot as plt 
from astropy.wcs import WCS
# from photutils import DAOStarFinder
from astropy.stats import mad_std
# from photutils import aperture_photometry, CircularAperture, CircularAnnulus
from sklearn.cluster import DBSCAN
from numpy import genfromtxt
%matplotlib inline

# --- plot parameters --- 
plt.rcParams["figure.figsize"] = (15,10)
plt.rc('font', family = 'serif', serif = 'cmr10') 
plt.rcParams.update({'font.size': 22})

## Master Bias M91 

In [ ]:
# --- Directory Change --- 
# os.chdir(r'C:\Users\Owen\Dropbox\UCD - Physics Undergrad\3 - Third Year\PHYC3170 Advanced Labs\4. Astronomical Image Analysis\python_notebooks\lab_data\M91\bias')

# --- Data Import ---
image_list_bias_M91 = [fits.getdata(image) for image in glob('lab_data/M91/bias/B130311*')] # Obtains an average image across all frames 
master_bias_M91 = np.average(image_list_bias_M91, axis = 0)
master_bias_M91 = np.delete(master_bias_M91, np.s_[0:50], axis=1) # removing over-scan. 
mst_bias_std = np.std(master_bias_M91)

print(np.shape(master_bias_M91))
print(mst_bias_std)
print("Avg. count", master_bias_M91.mean())

In [ ]:
# --- Plotting Master Bias --- 

vmin = master_bias_M91.mean() - master_bias_M91.std()
vmax = master_bias_M91.mean() + master_bias_M91.std()

plt.imshow(master_bias_M91, cmap = 'magma', vmin = vmin , vmax = vmax)
# plt.title("Master Bias for M91")
plt.grid()
plt.colorbar()
plt.savefig('images/M91/M91_master_bias_B.png')
plt.show()

In [ ]:
# --- creating a .fit master flat file --- 

nmasterbiasB_M91_header = fits.PrimaryHDU(master_bias_M91)
nmasterbiasB_M91_header.writeto('M91\masterbias_M91', overwrite = True)

## Master Flat M91 B Filter 

In [ ]:
# --- Directory Change --- 
#os.chdir(r'C:\Users\Owen\Dropbox\UCD - Physics Undergrad\3 - Third Year\PHYC3170 Advanced Labs\4. Astronomical Image Analysis\python_notebooks\lab_data\M91\flats\B')

# --- Data Import ---
image_list_flat_M91_B = [fits.getdata(image) for image in glob('lab_data/M91/flats/B/F130311*')] # - master_bias_M91 # Obtains an average image across all frames
master_flat_M91_B = np.average(image_list_flat_M91_B, axis = 0)
master_flat_M91_B = np.delete(master_flat_M91_B, np.s_[0:50], axis=1)

print(np.average(master_flat_M91_B)) #avg 
print(master_flat_M91_B.std()) #std 
print(np.average(master_flat_M91_B) - master_flat_M91_B.std(), np.average(master_flat_M91_B) + master_flat_M91_B.std()) #maxmin

In [ ]:
# --- Plotting Master Bias for B Filter --- 

vmin = master_flat_M91_B.mean() - master_flat_M91_B.std()
vmax = master_flat_M91_B.mean() + master_flat_M91_B.std()

plt.imshow(np.delete(master_flat_M91_B, np.s_[0:50], axis=1), cmap = 'magma', vmin = 25000, vmax = 27000)
# plt.title("Master Flat M91 (B)")
# plt.gca().invert_yaxis() # flips y axis 
plt.grid()
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_B.png')
plt.show()

In [ ]:
# --- Normalising the B filter flat --- 

b_m91_flat_avg = np.average(master_flat_M91_B[824:1224, 849:1249]) # --- average from the center of the flat frame --- 
print("Avg. of the sample", b_m91_flat_avg)

master_flat_M91_B_norm = master_flat_M91_B/b_m91_flat_avg

# --- Plotting the norm master flat for the B filter --- 

vmin = master_flat_M91_B_norm.mean() - master_flat_M91_B_norm.std()
vmax = master_flat_M91_B_norm.mean() + master_flat_M91_B_norm.std()

plt.imshow(np.delete(master_flat_M91_B_norm, np.s_[0:50], axis=1), cmap = 'magma', vmin = vmin, vmax = vmax)
# plt.title("Normalized Master Flat M91 (B)")
plt.grid()
# plt.gca().invert_yaxis() # flips y axis 
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_norm_B.png')
plt.show()

In [ ]:
# --- Plotting Master Bias for B Filter Zoomed  --- 


vmin = master_flat_M91_B.mean() - master_flat_M91_B.std()
vmax = master_flat_M91_B.mean() + master_flat_M91_B.std()

plt.imshow(master_flat_M91_B, cmap = 'magma', vmin = 22000, vmax = 30000)
# plt.title("Master Flat M91 Center Sample (B)")
plt.xlim(824, 1224)
plt.ylim(849, 1249)
# plt.gca().invert_yaxis() # flips y axis 
plt.grid()
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_center_zoom_B.png')
plt.show()

In [ ]:
# --- creating a .fit master flat file --- 

nmasterflatB_M91_header = fits.PrimaryHDU(master_flat_M91_B_norm)
nmasterflatB_M91_header.writeto('M91\masterflat_M91_B', overwrite = True)

In [ ]:
# --- Data Import ---
# image_list_flat_M91_HA = [fits.getdata(image) for image in glob('lab_data/M91/flats/B/F130311*')] # - master_bias_M91 # Obtains an average image across all frames
# master_flat_M91_HA = np.average(image_list_flat_M91_HA, axis = 0)
# master_flat_M91_HA = np.delete(master_flat_M91_B, np.s_[0:50], axis=1)

# print(np.average(master_flat_M91_B)) #avg 
# print(master_flat_M91_B.std()) #std 
# print(np.average(master_flat_M91_B) - master_flat_M91_B.std(), np.average(master_flat_M91_B) + master_flat_M91_B.std()) #maxmin

In [ ]:
# --- Plotting Master Flat for HA Filter --- 

vmin = master_flat_M91_HA.mean() - master_flat_M91_HA.std()
vmax = master_flat_M91_HA.mean() + master_flat_M91_HA.std()

plt.imshow(master_flat_M91_HA, cmap = 'magma', vmin = vmin, vmax = vmax)
# plt.title("Master Flat M91 (HA)")
plt.gca().invert_yaxis() # flips y axis 
plt.grid()
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_HA.png')
plt.show()

In [ ]:
# --- Plotting Master Flat for HA Filter Norm Area --- 

plt.imshow(master_flat_M91_HA, cmap = 'magma', vmin = 28000, vmax = 31000)
# plt.title("Master Flat M91 (HA)")
plt.gca().invert_yaxis() # flips y axis 
plt.grid()
plt.xlim(824, 1224)
plt.ylim(849, 1249)
plt.colorbar()
plt.show()
plt.savefig('images/M91/M91_master_flat_HA_zoomed.png')

In [ ]:
# --- Normalising the HA filter flat --- 

HA_m91_flat_avg = np.average(master_flat_M91_HA[824:1224, 849:1249]) # --- average from the center of the flat frame --- 
print("Avg. of the sample", HA_m91_flat_avg)

master_flat_M91_HA_norm = master_flat_M91_HA/HA_m91_flat_avg

# --- Plotting the norm master flat for the B filter --- 

plt.imshow(np.delete(master_flat_M91_HA_norm, np.s_[0:50], axis=1), cmap = 'magma', vmin = .95, vmax = 1.05)
# plt.title("Normalized Master Flat M91 (HA)")
plt.grid()
plt.gca().invert_yaxis() # flips y axis 
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_HA_Norm.png')
plt.show()

## Master Flat M91 V Filter 

In [ ]:
# --- Data Import ---
image_list_flat_M91_V = [fits.getdata(image) for image in glob('lab_data/M91/flats/V/*')] # - master_bias_M91 # Obtains an average image across all frames
master_flat_M91_V = np.average(image_list_flat_M91_V, axis = 0)
master_flat_M91_V = np.delete(master_flat_M91_V, np.s_[0:50], axis=1)

print(np.average(master_flat_M91_V))

In [ ]:
# --- Plotting Master Flat for V Filter --- 

vmin = master_flat_M91_V.mean() - master_flat_M91_V.std()
vmax = master_flat_M91_V.mean() + master_flat_M91_V.std()

plt.imshow(np.delete(master_flat_M91_V, np.s_[0:50], axis=1), cmap = 'magma', vmin = vmin, vmax = vmax)
# plt.title("Master Flat M91 (V)")
plt.gca().invert_xaxis() # flips y axis 
plt.grid()
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_V.png')
plt.show()

In [ ]:
# --- Normalising the V filter flat --- 

V_m91_flat_avg = np.average(master_flat_M91_V[824:1224, 849:1249]) # --- average from the center of the flat frame --- 
print("Avg. of the sample", V_m91_flat_avg)

master_flat_M91_V_norm = master_flat_M91_V/V_m91_flat_avg

vmin = master_flat_M91_V_norm.mean() - master_flat_M91_V_norm.std()
vmax = master_flat_M91_V_norm.mean() + master_flat_M91_V_norm.std()

# --- Plotting the norm master flat for the B filter --- 

plt.imshow(master_flat_M91_V_norm, cmap = 'magma', vmin = .95, vmax = 1.05)
# plt.title("Normalized Master Flat M91 (V)")
plt.grid()
plt.gca().invert_xaxis() # flips y axis 
plt.colorbar()
plt.savefig('images/M91/M91_master_flat_norm_V.png')
plt.show()

## Reduction of Objects in M91 

In [ ]:
# os.chdir(r'C:\Users\Owen\Dropbox\UCD - Physics Undergrad\3 - Third Year\PHYC3170 Advanced Labs\4. Astronomical Image Analysis\python_notebooks')
object_B_data = [fits.getdata(image) for image in glob('lab_data/M91/object/B/O130311*')]
print(np.shape(object_B_data))

In [ ]:
object_B = glob('lab_data/M91/object/B/O130311*')
object_B_pixels = []

mb_r = fits.getdata('M91\masterbias_M91')
print(np.shape(mb_r))

for file in object_B:
    object_BB = fits.open(file) 
    object_B_pixels = object_BB[0].data #getting pixel data
    object_B_subtracted = np.delete(object_B_pixels, np.s_[0:50], axis = 1) - mb_r #subtracting reduced masterbias 
    object_B_reduced =  object_B_subtracted/master_flat_M91_B_norm #diving by normalised master_flat 
    object_B_pixels = object_B_reduced
    newheader = fits.PrimaryHDU(object_B_reduced)
    newfilename = "M91\object_B_reduced_" + file.split('/')[-1]
    newheader.writeto(newfilename, overwrite = True)

In [ ]:
object_B_reduced_data = [fits.getdata(image) for image in glob('object_B_reducedB/O130311*')]
np.shape(object_B_reduced_data)

In [ ]:

# --- Looking at the reduced image output before proceeding further 

test_print = glob('object_B_reducedB/O130311*')

for file in test_print:
    std = (np.std(object_B_reduced))
    vmin = object_B_reduced.mean() - object_B_reduced.std()
    vmax = object_B_reduced.mean() + object_B_reduced.std()

    plt.imshow(object_B_pixels , vmin = vmin, vmax = vmax, cmap = 'magma') 
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

### Repeating for the H $$\alpha$$ filter 

In [ ]:
object_HA_data = [fits.getdata(image) for image in glob('lab_data/M91/object/Halpha_NII/O130311*')]
object_HA = glob('lab_data/M91/object/Halpha_NII/O130311*')
object_HA_pixels = []
mb_r = fits.getdata('M91\masterbias_M91')

for file in object_HA:
    object_HHA = fits.open(file) # stores fit file 
    object_HA_pixels = object_HHA[0].data # access of pixel data 
    object_HA_subtracted = np.delete(object_HA_pixels, np.s_[0:50], axis = 1) - mb_r #
    object_HA_reduced =  object_HA_subtracted/master_flat_M91_HA_norm #
    object_HA_pixels = object_HA_reduced
    newheader = fits.PrimaryHDU(object_HA_reduced)
    newfilename = "M91\object_HA_reduced_" + file.split('/')[-1]
    newheader.writeto(newfilename, overwrite = True)

In [ ]:

# --- Looking at the reduced image output before proceeding further 
object_HA_reduced = [fits.getdata(image) for image in glob('M91/object_HA_reduced_Halpha_NII/O130311*')]
test_print = glob('M91/object_HA_reduced_Halpha_NII/O130311*')

for file in test_print:
    std = (np.std(object_HA_reduced))
    vmin = 1*std
    vmax = 3*std
    plt.imshow(object_HA_pixels , cmap = 'magma', vmin = vmin, vmax = vmax) 
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
object_HA_reduced = [fits.getdata(image) for image in glob('M91/object_HA_reduced_Halpha_NII/O130311*')]
print(np.shape(object_HA_reduced))
master_HA = np.average(object_HA_reduced, axis = 0)

vmin = master_HA.mean() - master_HA.std()
vmax = master_HA.mean() + master_HA.std()


plt.imshow(master_HA , cmap = 'magma', vmin = vmin, vmax = vmax) 
plt.colorbar()
# plt.gca().invert_yaxis()
plt.savefig('images/M91/M91_combined_HA.png')
# plt.show()

### For V filter 

In [ ]:
object_V_data = [fits.getdata(image) for image in glob('lab_data/M91/object/V/O130311*')]
object_V = glob('lab_data/M91/object/V/O130311*')
object_V_pixels = []

for file in object_V:
    object_HV = fits.open(file) 
    object_V_pixels = object_HV[0].data 
    object_V_subtracted = np.delete(object_V_pixels, np.s_[0:50], axis=1) - mb_r #
    object_V_reduced =  object_V_subtracted/master_flat_M91_V_norm #
    object_V_pixels = object_V_reduced
    newheader = fits.PrimaryHDU(object_V_reduced)
    newfilename = "M91\object_V_reduced_" + file.split('/')[-1]
    newheader.writeto(newfilename, overwrite = True)

In [ ]:
# --- Looking at the reduced image output before proceeding further 
vmin = object_V_pixels.mean() - object_V_pixels.std() 
vmax = object_V_pixels.mean() + object_V_pixels.std() 

test_print = glob('M91/object_V_reduced_V/O130311*')

for file in test_print:
    std = (np.std(object_V_reduced))
    vmin = 1*std
    vmax = 3*std
    plt.imshow(object_V_pixels, cmap = 'magma', vmin = vmin, vmax = vmax) 
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
filterV = [fits.getdata(image) for image in glob('M91/object_V_reduced_V/O130311*')]
filterV_da = np.average(filterV, axis = 0)

vmin = filterV_da.mean() - filterV_da.std()
vmax = filterV_da.mean() + filterV_da.std()


plt.imshow(filterV_da , cmap = 'magma', vmin = vmin, vmax = vmax) 
plt.colorbar()
# plt.gca().invert_yaxis()
plt.savefig('images/M91/M91_combined_V.png')
#plt.show()

In [ ]:
filterB = [fits.getdata(image) for image in glob('M91/object_B_reduced_B/O130311*')]
filterB_da = np.average(filterV, axis = 0)

vmin = filterB_da.mean() - filterB_da.std()
vmax = filterB_da.mean() + filterB_da.std()


plt.imshow(filterB_da , cmap = 'magma', vmin = vmin, vmax = vmax) 
plt.colorbar()
# plt.gca().invert_yaxis()
plt.savefig('images/M91/M91_combined_B.png')

## Combining images

In [ ]:
filterB = [fits.getdata(image) for image in glob('M91/object_B_reduced_B/O130311*')]
filterHA = [fits.getdata(image) for image in glob('M91/object_HA_reduced_Halpha_NII/O130311*')]
filterV = [fits.getdata(image) for image in glob('M91/object_V_reduced_V/O130311*')]

print(np.shape(filterB), np.shape(filterHA), np.shape(filterV))

combined_array = np.concatenate((filterB, filterHA, filterV), axis = 0)
print(np.shape(combined_array))

master_image = np.average(combined_array, axis = 0)


newfilename = "master_images/master_image_M91"
print(newfilename)
newheader.writeto(newfilename, overwrite = True)

In [ ]:

vmin = master_image.mean() - master_image.std() 
vmax = master_image.mean() + master_image.std() 

print(vmin, vmax)
plt.imshow(np.delete(master_image, np.s_[0:50], axis=1), cmap = 'magma', vmin = vmin, vmax = vmax) 
plt.colorbar()
plt.gca().invert_xaxis()
plt.show()

## Angular Momentum Analysis for $M_{91}$ 

In [ ]:
M91_master_image = fits.open('master_images/master_image_M91')
M91_master_data = fits.getdata('master_images/master_image_M91')
M91_master_data = np.delete(M91_master_data, np.s_[0:50], axis = 1)
M91_wcs = WCS(M91_master_image[0].header)

# print(M91_wcs)
print(M91_master_data.mean())
print(M91_master_data.std())

# --- Projection Plot --- 

vmin = M91_master_data.mean() - M91_master_data.std() 
vmax = M91_master_data.mean() + M91_master_data.std() 

# plt.subplot(projection = M91_wcs)
plt.imshow(M91_master_image[0].data, cmap = 'magma', vmin = vmin, vmax = vmax) 
plt.colorbar()
plt.grid(True, which = 'major', alpha = .5)
# plt.minorticks_on()
# plt.grid(True, which = 'minor', alpha = .2)
plt.xlim(600, 1450)
plt.ylim(400, 1400)
plt.gca().invert_xaxis()
plt.savefig('images/M91/M91_plain-master-zoom.png')
plt.show()

plt.imshow(M91_master_image[0].data, cmap = 'magma', vmin = vmin, vmax = vmax) 
plt.colorbar()
plt.grid(True, which = 'major', alpha = .5)
# plt.minorticks_on()
# plt.grid(True, which = 'minor', alpha = .2)
# plt.gca().invert_xaxis()
# plt.show()
plt.savefig('images/M91/M91_plain-master.png')



In [ ]:
baseline = np.mean(M91_master_image[0].data[1800:2000, 1800:2000] + 5*(M91_master_image[0].data[1800:2000, 1800:2000].std())) #creating a standard to compare images against.

M91_master_data = fits.getdata('master_images/master_image_M91')

bright_sources = np.where(M91_master_data > baseline)
x_pos = bright_sources[0]
y_pos = bright_sources[1]

combined_positions = np.vstack((x_pos, y_pos)).T # transposing so axis match images. 
print(np.shape(combined_positions))

empty_image = np.zeros((np.shape(M91_master_image[0].data)[1], np.shape(M91_master_image[0].data)[1])) # empty array the size of the image so position data can be overlayed. 
print(np.shape(empty_image))

for i in combined_positions:
    empty_image[i[0], i[1]] = 1 # loop to add positions to 'blank' image. 

bright_isolated_image = empty_image

# --- Plotting --- 

plt.grid(True, which='major', alpha=0.5)
plt.minorticks_on()

plt.grid(True, which='minor', alpha=0.2)

plt.imshow(bright_isolated_image, cmap = 'magma', vmin=0, vmax=1)
plt.xlim(600, 1500)
plt.ylim(400, 1400)
plt.colorbar()
plt.gca().invert_xaxis()
plt.savefig('images/M91/M91_1pixel-heatmap.png')


# --- Plotting onto White Backghround --- 

plt.figure(figsize = (10, 10))
plt.grid(True, which='major', alpha = 0.5)
plt.minorticks_on()
plt.gca().invert_xaxis()
# plt.xlim(600, 1500)
# plt.ylim(350, 1400)
plt.grid(True,  which = 'minor', alpha=0.5)
plt.scatter(y_pos, x_pos, s = .5)
plt.savefig('images/M91/M91_1pixel-master.png')

In [ ]:
# %%time

baseline = np.mean(M91_master_image[0].data[1800:2000, 1800:2000] + 3*(M91_master_image[0].data[1800:2000, 1800:2000].std())) #creating a standard to compare images against. from corner dark region with a lack of sources.  
# baseline = 423
# --- Obeserving Areas Para-meters --- 

# -> semi-major axis midpoint between periapsis and apoapsis. the midpoint on a line of x-axis with a certain intensity. 
# -> semi-minor axis. the midpoint on a line of y-axis with a certain intensity.

mst_image_data = M91_master_image[0].data
# print(mst_image_data.mean())

y_start = 0 
y_limit = 0 
pixel_increment = 15 # pixel increments 
x_array = []
y_array = []
a = 1
counter = 0


# print(np.shape(mst_image_data)[1]/pixel_increment)
# test = np.arange(0, np.shape(mst_image_data)[1], pixel_increment)
# print(test.max()/pixel_increment)

while y_limit < np.shape(mst_image_data)[1]:
    for i in range(0, np.shape(mst_image_data)[1], 1):
        counts = mst_image_data[i*pixel_increment:i*pixel_increment + pixel_increment, y_start:y_start + pixel_increment]
        counter += 1 
        # print(counter)
        if counts.mean() > baseline:
            # x_pos_c = i*pixel_increment
            x_pos_c = (i*pixel_increment + (i*pixel_increment + pixel_increment))/2 #center of box for easy viewing + calculations.   
            # print(x_pos_c)
            x_array = np.append(x_array, x_pos_c)

            # y_pos_c = y_start
            y_pos_c = (y_start + (y_start + pixel_increment))/2 
            # print(y_pos_c)
            y_array = np.append(y_array, y_pos_c)

        # print(y_limit)
        y_limit += pixel_increment

        if counter > 2047: 
            y_start += pixel_increment 
            y_limit = 0
            counter = 0  
            a += 1
            # print(y_start)
            if y_start > 2000:
                y_limit = np.shape(mst_image_data)[1]
                break
        # break  
        
# print(np.shape(M91_master_image[0].data)[1])
print(len(x_array), len(y_array)) 
# print(x_array)

In [ ]:
def brightness_function(pixel_increment, image_data, baseline): 
    # --- constant definitions and variable set up: --- 
    y_start = 0 
    y_limit = 0
    counter = 0

    x_array = []
    y_array = []

    while y_limit < np.shape(image_data)[1]:
        for i in range(0, np.shape(image_data)[1], 1):
            counts = mst_image_data[i*pixel_increment:i*pixel_increment + pixel_increment, y_start:y_start + pixel_increment]
            counter += 1 
            if counts.mean() > baseline:
                x_pos_c = (i*pixel_increment + (i*pixel_increment + pixel_increment))/2 #center of box for easy viewing + calculations.   
                x_array = np.append(x_array, x_pos_c)

                y_pos_c = (y_start + (y_start + pixel_increment))/2 # doing the same for the y-pos
                # print(y_pos_c)
                y_array = np.append(y_array, y_pos_c)

            # y_limit += pixel_increment

            if counter > np.shape(image_data)[1] + 1: 
                y_start += pixel_increment 
                y_limit = 0
                # print(y_start)
                counter = 0  
                if y_start > 2000:
                    y_limit = np.shape(mst_image_data)[1]
                    return(x_array, y_array)
    print("The number of", pixel_increment, 'x', pixel_increment, "that exceeded the baseline value of", baseline, " counts, was found to be", len(x_array))   
    

In [ ]:
t_20 = brightness_function(15, mst_image_data, baseline)
t_10 = brightness_function(10, mst_image_data, baseline)

# Output = [t_20[0], t_20[1]]
# dataset = pd.DataFrame({'X': Output[0], 'Y': Output[1]})
# dataset.to_csv('saved_data/15_pixels.csv')

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(y_array, x_array, s = 1.5)
plt.grid()
# plt.gca().invert_yaxis()
# plt.gca().invert_xaxis()

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(y_pos, x_pos, color = 'red', s = .2, label = '1 x 1')
plt.scatter(t_10[1], t_10[0], color = 'blue', s = 1.5,  label = '10 x 10')
plt.scatter(t_20[1], t_20[0], color = 'green', s = 2, label = '15 x 15')
plt.grid()
plt.legend()
plt.gca().invert_xaxis()
plt.savefig('images/M91/M91_grid_compare.png')


In [ ]:
vmin = M91_master_data.mean() - M91_master_data.std() 
vmax = M91_master_data.mean() + M91_master_data.std() 

# plt.plot(y_pos, x_pos, '.')

plt.scatter(t_10[1], t_10[0], s = 4)
plt.scatter(t_20[1], t_20[0], s = 4, color = 'purple')
plt.imshow(M91_master_image[0].data, cmap = 'magma', vmin = vmin, vmax = vmax) 
# plt.gca().invert_xaxis()

# plt.imshow(bright_isolated_image, cmap = 'magma', vmin=0, vmax=1)
plt.colorbar()
plt.grid(True, which = 'major', alpha = .5)

plt.savefig('images/M91/M91_overlay_compare.png')

In [ ]:
feature_space = pd.read_csv("saved_data/15_pixels.csv", header = None, sep=",")
# print(feature_space.tail)
data = feature_space.iloc[:, 0:2].values
# print(data)

model = DBSCAN(eps = 40, min_samples  =  8).fit(data)

csv = genfromtxt("saved_data/15_pixels.csv", delimiter = ",")

# Slicing array
# X = csv[:, 0]
# Y = csv[:, 1]

X = x_array
Y = y_array

print(X.max())



var = model.labels_ 
# print(var)

plt.figure(figsize=(10, 10))
plt.scatter(Y, X, c = var, marker='o', s = 5)
plt.grid()
plt.gca().invert_xaxis()

plt.savefig('images/M91/M91_DBSCAN_15.png')

In [ ]:
Xn = []
Yn = []

for i in range(0, len(var)):
    if var[i] == 0:
        Xn = np.append(Xn, X[i])
        Yn = np.append(Yn, Y[i])
    
Xn = Xn[0:-2]
Yn = Yn[0:-2]

pos = [Xn, Yn]

# print(np.count_nonzero(var == 1))
# print(len(Xn), len(Yn))
plt.figure(figsize = (10, 10))
print(Yn.max(), Xn.max())
plt.scatter(Yn, Xn)

In [ ]:
def rotate_via_numpy(xy, radians):
    """Use numpy to build a rotation matrix and take the dot product."""
    x, y = xy
    c, s = np.cos(radians), np.sin(radians)
    j = np.matrix([[c, s], [-s, c]])
    m = np.dot(j, [x, y])

    return float(m.T[0]), float(m.T[1])

In [ ]:
# --- Rotating Data Points --- 

print(np.shape(pos))

radians = 3.14159
new_pos = []

tt = []

for i in range(0, len(pos[1])):
    x = pos[0][i]
    y = pos[1][i]
    c, s = np.cos(radians), np.sin(radians)
    j = np.matrix([[c, s], [-s, c]])
    m = np.dot(j, [x, y])

    NP = float(m.T[0]), float(m.T[1])
    NP = np.asarray(NP) 
    new_pos.append(NP)

new_x = []
new_y = []

for i in range(0, 472):
    # print(new_pos[i][0])
    new_x.append(new_pos[i][0])
    new_y.append(new_pos[i][1])

new_x = np.array(new_x)
new_y = np.array(new_y)

# --- semi-major-axis --- 

ma = (new_y.max() - new_y.min())
mm = (new_x.max() - new_x.min())

ma_arcsec = ma*.304
print('Semi-major-axis, pixels:', ma)
print('Semi-major-axis, areseconds:', ma*.304)
print('Semi-major-axis, arcmins:', ma*.304/60)

mm_arcsec = ma*.304
print('Semi-minor-axis, pixels:', mm)
print('Semi-minor-axis, areseconds:', mm*.304)
print('Semi-minor-axis, arcmins:', (mm*.304)/60)
# --- Correcting --- 

def parsecs(arcsecs, distance):
    return (arcsecs/206625)*distance

distance = 15e3 # \pm .8 

print('Major-axis, KPC:', parsecs(ma_arcsec, distance))
print('minor-axis, KPC:', parsecs(mm_arcsec, distance))

ma_degrees = ma_arcsec/3600
mm_degrees = mm_arcsec/3600

def view_correction(distance, angle, i):
    return (distance*np.tan(angle))/np.cos(i)

i = 36.9


# plt.axhline(y = new_y.max())
# plt.axhline(y = new_y.min())
# plt.axvline(x = new_x.max())
# plt.axvline(x = new_x.min())
# plt.scatter(new_x, new_y)

In [ ]:
vmin = M91_master_data.mean() - M91_master_data.std() 
vmax = M91_master_data.mean() + M91_master_data.std() 

# plt.plot(y_pos, x_pos, '.')
plt.scatter(Yn, Xn, s = 3)
plt.imshow(M91_master_image[0].data, cmap = 'magma', vmin = vmin, vmax = vmax) 

# plt.imshow(bright_isolated_image, cmap = 'magma', vmin=0, vmax=1)
plt.colorbar()
plt.grid(True, which = 'major', alpha = .7)


plt.savefig('images/M91/M91_DBSCAN_overlay.png')